# Imports

In [ ]:
!pip install --upgrade pyalgotrading

In [1]:
from pyalgotrading.algobulls import AlgoBullsConnection

# Connection

In [2]:
connection = AlgoBullsConnection()

In [3]:
connection.get_token_url()

Please login to this URL to get your unique token: https://app.algobulls.com/settings?section=developerOptions


In [4]:
API_TOKEN = "39394ea25014909e2c45b4210fe78d77f5440470"
connection.set_access_token(API_TOKEN)

Access token is valid.


# Strategy Creation

## Import Strategy from pyaglostrategypool

In [5]:
! wget -O rsi_macd_crossover.py https://raw.githubusercontent.com/algobulls/pyalgostrategypool/master/pyalgostrategypool/rsi_macd_crossover/_strategy.py
! sed -i '1s/^/from pyalgotrading.strategy import StrategyBase\n/' rsi_macd_crossover.py

--2024-01-13 23:07:19--  https://raw.githubusercontent.com/algobulls/pyalgostrategypool/master/pyalgostrategypool/rsi_macd_crossover/_strategy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3704 (3.6K) [text/plain]
Saving to: ‘rsi_macd_crossover.py’

rsi_macd_crossover. 100%[===================>]   3.62K  --.-KB/s    in 0s      

2024-01-13 23:07:19 (46.6 MB/s) - ‘rsi_macd_crossover.py’ saved [3704/3704]



In [6]:
from rsi_macd_crossover import RSIMACDCrossover as strategy_cls

In [7]:
response = connection.create_strategy(strategy_cls, overwrite=True)
response

{'message': 'Strategy saved successfully',
 'cstcId': 136419,
 'strategyId': '3cc07957398b4ffb94e12a68a45e4ea0'}

In [8]:
strategy = response['strategyId']

# Strategy Testing

## Instruments Searching (optional)

In [9]:
instrument = connection.search_instrument('HEROMOTOCO', exchange='NSE')
instrument

[{'id': 28, 'value': 'NSE:HEROMOTOCO'},
 {'id': 1672355, 'value': 'NSE:HEROMOTOCO.AF'},
 {'id': 1672479, 'value': 'NSE:HEROMOTOCO.BL'},
 {'id': 2316395, 'value': 'NSE:HEROMOTOCO.L1'},
 {'id': 2041177, 'value': 'NSE:HEROMOTOCO.R1'},
 {'id': 2041176, 'value': 'NSE:HEROMOTOCO.V1'},
 {'id': 24609, 'value': 'NSE:HEROMOTOCO20NOV2150CE'},
 {'id': 24610, 'value': 'NSE:HEROMOTOCO20NOV2150PE'},
 {'id': 24611, 'value': 'NSE:HEROMOTOCO20NOV2200CE'},
 {'id': 24612, 'value': 'NSE:HEROMOTOCO20NOV2200PE'}]

## Setup Parameters

In [10]:
parameters = {
    'OVERBOUGHT_VALUE': 85,
    'OVERSOLD_VALUE': 15,
    'TIMEPERIOD_FAST': 12,
    'TIMEPERIOD_RSI': 3,
    'TIMEPERIOD_SIGNAL': 9,
    'TIMEPERIOD_SLOW': 26
}

In [11]:
initial_virtual_funds = 1000000000     # in Rupees

## Backtesting

### Start

In [12]:
connection.backtest(
    strategy=strategy, 
    start='2021-08-01 09:15 +0530', 
    end='2023-07-31 15:30 +0530', 
    instrument='NSE:HEROMOTOCO', 
    lots=2,
    parameters=parameters, 
    candle='1 hour',
    initial_funds_virtual=initial_virtual_funds

)


Starting the strategy 'RSI MACD Crossover' in BACKTESTING mode...
╒═════════════════════════╤═══════════════════════════╕
│ Config                  │ Value                     │
╞═════════════════════════╪═══════════════════════════╡
│ Strategy Name           │ RSI MACD Crossover        │
├─────────────────────────┼───────────────────────────┤
│ Trading Type            │ BACKTESTING               │
├─────────────────────────┼───────────────────────────┤
│ Instrument(s)           │ ['NSE:HEROMOTOCO']        │
├─────────────────────────┼───────────────────────────┤
│ Quantity/Lots           │ 2                         │
├─────────────────────────┼───────────────────────────┤
│ Start Timestamp         │ 2021-08-01 09:15:00+05:30 │
├─────────────────────────┼───────────────────────────┤
│ End Timestamp           │ 2023-07-31 15:30:00+05:30 │
├─────────────────────────┼───────────────────────────┤
│ Parameters              │ {'OVERBOUGHT_VALUE': 85,  │
│                         │  'OVERSOL

### Status

In [13]:
connection.get_backtesting_job_status(strategy)

{'message': 'STARTED', 'status': 2}

### Logs

In [ ]:
logs = connection.get_backtesting_logs(strategy)

Execution Progress:   0%|          | 0/63008100.0 [00:00<?]

In [ ]:
print(logs)

### Stop

In [ ]:
connection.stop_backtesting_job(strategy)

### Profit and Loss Reports

In [ ]:
pnl_reports = connection.get_backtesting_report_pnl_table(strategy)
pnl_reports

### Statistics Reports

#### Statistics

In [ ]:
connection.get_backtesting_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_backtesting_report_statistics(strategy, report='full', html_dump=True )

### Order History

In [ ]:
order_history = connection.get_backtesting_report_order_history(strategy)
print(order_history)

---

## Papertrading

### Start

In [ ]:
connection.papertrade(
    strategy=strategy, 
    start='9:15 +0530', 
    end='15:00 +0530',  
    instruments='NSE:ADANIPOWER', 
    lots=5,
    parameters=parameters, 
    candle='1 minute',
)

### Status

In [ ]:
connection.get_papertrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_papertrading_logs(strategy)
print(logs)

### Stop

### Profit and Loss Reports (Paper Trading)

In [ ]:
pnl_reports = connection.get_papertrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Paper Trading)

#### Statistics

In [ ]:
connection.get_papertrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_papertrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)

### Order History (Paper Trading)

In [ ]:
order_history = connection.get_papertrading_report_order_history(strategy)
print(order_history)

---